In [4]:
import os
import math
import numpy as np
import tensorflow as tf
from PIL import Image
import time

In [15]:
VGG_MEAN = [103.939, 116.779, 123.68]

class VGGNet:
    """构建VGG-16网络结构， 读取模型参数"""
    def __init__(self, data_dict):
        self.data_dict = data_dict
    
    def get_conv_filter(self, name):
        return tf.constant(self.data_dict[name][0], name='conv')
    def get_fc_weight(self, name):
        return tf.constant(self.data_dict[name][0], name='fc')
    def get_bias(self, name):
        return tf.constant(self.data_dict[name][1], name='bias')
    def conv_layer(self, x, name):
        """构建卷积层"""
        with tf.name_scope(name):
            conv_w = self.get_conv_filter(name)
            conv_b = self.get_bias(name)
            h = tf.nn.conv2d(x, conv_w, [1, 1, 1, 1], padding='SAME')
            h = tf.nn.bias_add(h, conv_b)
            h = tf.nn.relu(h)
            return h
    
    def pooling_layer(self, x, name):
        """构建池化层"""
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)
    
    def fc_layer(self, x, name, activation=tf.nn.relu):
        """构建全连接层"""
        with tf.name_scope(name):
            fc_w = self.get_fc_weight(name)
            fc_b = self.get_bias(name)
            h = tf.matmul(x, fc_w)
            h = tf.nn.bias_add(h, fc_b)
            if activation is None:
                return h
            else:
                return activation(h)
    
    def flatten_layer(self, x, name):
        with tf.name_scope(name):
            x_shape = x.get_shape().as_list()
            dim = 1
            for d in x_shape[1:]:
                dim *= d
            x = tf.reshape(x, [-1, dim])
            return x
        
    def build(self, x_rgb):
        """构建VGG16网络结构
        parameters:
        - x_rgb: [1, 224, 224, 3]
        """
        start_time = time.time()
        print('构建开始 building model ...')
        
        r, g, b = tf.split(x_rgb, [1, 1, 1], axis=3)
        x_bgr = tf.concat([b - VGG_MEAN[0], 
                           g - VGG_MEAN[1], 
                           r - VGG_MEAN[2]], 
                         axis=3)
        
        assert x_bgr.get_shape().as_list()[1:] == [224, 224, 3]
        
        self.conv1_1 = self.conv_layer(x_bgr, b'conv1_1')
        self.conv1_2 = self.conv_layer(self.conv1_1, b'conv1_2')
        self.pool1 = self.pooling_layer(self.conv1_2, 'pool1')
        
        self.conv2_1 = self.conv_layer(self.pool1, b'conv2_1')
        self.conv2_2 = self.conv_layer(self.conv2_1, b'conv2_2')
        self.pool2 = self.pooling_layer(self.conv2_2, 'pool2')
        
        self.conv3_1 = self.conv_layer(self.pool2, b'conv3_1')
        self.conv3_2 = self.conv_layer(self.conv3_1, b'conv3_2')
        self.conv3_3 = self.conv_layer(self.conv3_2, b'conv3_3')
        self.pool3 = self.pooling_layer(self.conv3_3, 'pool3')
        
        self.conv4_1 = self.conv_layer(self.pool3, b'conv4_1')
        self.conv4_2 = self.conv_layer(self.conv4_1, b'conv4_2')
        self.conv4_3 = self.conv_layer(self.conv4_2, b'conv4_3')
        self.pool4 = self.pooling_layer(self.conv4_3, 'pool4')
        
        self.conv5_1 = self.conv_layer(self.pool4, b'conv5_1')
        self.conv5_2 = self.conv_layer(self.conv5_1, b'conv5_2')
        self.conv5_3 = self.conv_layer(self.conv5_2, b'conv5_3')
        self.pool5 = self.pooling_layer(self.conv5_3, 'pool5')
        
        self.flatten5 = self.flatten_layer(self.pool5, 'flatten5')
        
        self.fc6 = self.fc_layer(self.flatten5, b'fc6')
        self.fc7 = self.fc_layer(self.fc6, b'fc7')
        self.fc8 = self.fc_layer(self.fc7, b'fc8', activation=None)
        self.prob = tf.nn.softmax(self.fc8, name='prob')
        
        print('构建结束 building model finished: %4ds' % (time.time() - start_time))
        

In [18]:
vgg16_npy_pyth = 'vgg16.npy'
content_img_path = 'timg.jpg'
style_img_path = 'timg1.jpg'

num_steps = 100
learning_rate = 10

lambda_c = 0.1
lambda_s = 500

output_dir = './run_style_transfer'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

"""
data_dict = np.load('vgg16.npy',encoding='bytes').item()

vgg16_for_result = VGGNet(data_dict)
content = tf.placeholder(tf.float32, shape=[1, 224, 224, 3])
vgg16_for_result.build(content)"""

"\ndata_dict = np.load('vgg16.npy',encoding='bytes').item()\n\nvgg16_for_result = VGGNet(data_dict)\ncontent = tf.placeholder(tf.float32, shape=[1, 224, 224, 3])\nvgg16_for_result.build(content)"

In [21]:
def initial_result(shape, mean, stddev):
    """生成初始化图像"""
    initial = tf.truncated_normal(shape, mean=mean, stddev=stddev)
    return tf.Variable(initial)

def read_img(img_name):
    img = Image.open(img_name)
    np_img = np.array(img)
    np_img = np.asarray([np_img], dtype=np.int32)
    return np_img

result = initial_result((1, 224, 224, 3), 127.5, 20)
content_val = read_img(content_img_path)
style_val = read_img(style_img_path)

content = tf.placeholder(tf.float32, shape=[1, 224, 224, 3])
style = tf.placeholder(tf.float32, shape=[1, 224, 224, 3])

"""提取特征"""
data_dict = np.load('vgg16.npy',encoding='bytes').item()
vgg16_for_content = VGGNet(data_dict)
vgg16_for_style = VGGNet(data_dict)
vgg16_for_result = VGGNet(data_dict)

vgg16_for_content.build(content)
vgg16_for_style.build(style)
vgg16_for_result.build(result)

构建开始 building model ...
构建结束 building model finished:    2s
构建开始 building model ...
构建结束 building model finished:    2s
构建开始 building model ...
构建结束 building model finished:    1s
